In [25]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers, models, callbacks
import tensorflow as tf
from tensorflow import keras

def get_mnist_model():
    inputs = keras.Input(shape=(28*28,))
    features = layers.Dense(512, activation="relu")(inputs)
    features = layers.Dropout(0.5)(features)
    outputs = layers.Dense(10, activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model

(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28*28)).astype("float32")/255
test_images = test_images.reshape((10000,28*28)).astype("float32")/255
train_images,val_images = images[10000:], images[:10000]
train_labels,val_labels = labels[10000:], labels[:10000]

In [26]:
class LearningRateHandle(keras.callbacks.Callback):
     def on_epoch_end(self, epoch, logs):
        current_lr = tf.keras.backend.get_value(self.model.optimizer.lr)

        # 검증 손실 가져오기
        val_loss = logs.get('val_loss')
        
        if epoch == 0:
            print(f"Epoch {epoch+1}: 첫번째는 건너뛰기")
            return
        
        previous_val_loss = self.model.history.history['val_loss'][-1]

        # 검증 손실이 증가한 경우 학습률 감소
        if val_loss > previous_val_loss:
            new_lr = current_lr * 0.5  # 학습률 감소 비율
            tf.keras.backend.set_value(self.model.optimizer.lr, new_lr)
            print(f"\nEpoch {epoch+1}: 검증 손실 증가로 학습률을 줄입니다: {new_lr:.6f}")

In [27]:
model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.fit(train_images,train_labels,
          epochs=10,
          callbacks=[LearningRateHandle()],
          validation_data=(val_images, val_labels))

Epoch 1/10
1563/1563 [==============================] - 15s 8ms/step - loss: 0.2942 - accuracy: 0.9124 - val_loss: 0.1525 - val_accuracy: 0.9555
Epoch 1: 첫번째는 건너뛰기
Epoch 2/10
1563/1563 [==============================] - 13s 8ms/step - loss: 0.1644 - accuracy: 0.9538 - val_loss: 0.1231 - val_accuracy: 0.9674
Epoch 3/10
1563/1563 [==============================] - 12s 8ms/step - loss: 0.1386 - accuracy: 0.9640 - val_loss: 0.1200 - val_accuracy: 0.9714
Epoch 4/10
1563/1563 [==============================] - 12s 8ms/step - loss: 0.1232 - accuracy: 0.9686 - val_loss: 0.1133 - val_accuracy: 0.9727
Epoch 5/10
1563/1563 [==============================] - 12s 8ms/step - loss: 0.1144 - accuracy: 0.9715 - val_loss: 0.1113 - val_accuracy: 0.9755
Epoch 6/10
1563/1563 [==============================] - 12s 8ms/step - loss: 0.1085 - accuracy: 0.9735 - val_loss: 0.1124 - val_accuracy: 0.9747

Epoch 6: 검증 손실 증가로 학습률을 줄입니다: 0.000500
Epoch 7/10
1563/1563 [==============================] - 12s 8ms/step - 

### 회고록 
- 첫번째 에포크 때 오류가 게속나서 확인하였습니다. 

-이전 로스값과 현재 로스값을 비교해서 하는 동작인데 첫번째 에포크를 뛰어넘어야지 오류가 나지 않는것을 확인하여 코드를 수정하였습니다.

- 시간이 부족해서 조금 더 요소들을 추가하지 못한것이 아쉽지만 callback에 대해서 이해해서 좋았습니다.


### 피드백
-loss값이 내려간다고 해서 학습률을 내리면 무조건 개선되는지 여부에 대해서는 확실하지가 않다.

-바로 값을 바꾸는것보다 patience를 삽입해서 하는것이 좋을 것 같다.

-간단하게 작성해주셔서 이해하기가 빨랐습니다.